In [1]:
import numpy as np
import pandas as pd

# =========================
# 0) Wczytanie danych
# =========================
df = pd.read_csv("pacjenci_demo_system_ekspertowy.csv")
df.head()


# =========================
# 1) Reguły klasyczne (binarne)
# =========================
classic_rules = [
    {
        "name": "R1_HTN",
        "if": lambda p: p["systolic_bp"] >= 140 and p["diastolic_bp"] >= 90,
        "then": "Hypertension",
        "weight": 0.9
    }
]

def expert_system_classic(patient, rules):
    conclusions = {}
    fired = []

    for rule in rules:
        if rule["if"](patient):
            d = rule["then"]
            w = rule["weight"]
            conclusions[d] = max(conclusions.get(d, 0), w)
            fired.append({"rule": rule["name"], "then": d, "weight": w})

    return conclusions, fired


def counterfactual_classic_htn(patient, sbp_thr=140, dbp_thr=90):
    sbp = patient["systolic_bp"]
    dbp = patient["diastolic_bp"]

    diagnosed = (sbp >= sbp_thr) and (dbp >= dbp_thr)

    # Minimalna zmiana aby "przełamać" AND: wystarczy obniżyć SBP lub DBP poniżej progu
    if diagnosed:
        delta_sbp = max(0, sbp - (sbp_thr - 1))
        delta_dbp = max(0, dbp - (dbp_thr - 1))

        # wybór najmniejszej zmiany
        if delta_sbp <= delta_dbp:
            suggestion = {
                "action": "lower_sbp",
                "delta": int(delta_sbp),
                "target_value": int(sbp_thr - 1),
                "message": f"Aby zmienić decyzję klasyczną na 'brak HTN', należałoby obniżyć SBP o ok. {int(delta_sbp)} (do {sbp_thr-1})."
            }
        else:
            suggestion = {
                "action": "lower_dbp",
                "delta": int(delta_dbp),
                "target_value": int(dbp_thr - 1),
                "message": f"Aby zmienić decyzję klasyczną na 'brak HTN', należałoby obniżyć DBP o ok. {int(delta_dbp)} (do {dbp_thr-1})."
            }
    else:
        # Pacjent poniżej progów: pokazanie marginesu bezpieczeństwa
        margin_sbp = (sbp_thr - sbp)
        margin_dbp = (dbp_thr - dbp)
        suggestion = {
            "action": "none",
            "delta": 0,
            "target_value": None,
            "message": f"Decyzja klasyczna nie wskazuje HTN. Margines do progów: SBP {margin_sbp:.0f}, DBP {margin_dbp:.0f}."
        }

    return {"diagnosed": diagnosed, "suggestion": suggestion}

def counterfactual_classic_threshold_change(patient, sbp_thr=140, dbp_thr=90):
    sbp = float(patient["systolic_bp"])
    dbp = float(patient["diastolic_bp"])

    diagnosed = (sbp >= sbp_thr) and (dbp >= dbp_thr)

    # Minimalna zmiana progu, aby zmienić decyzję:
    # - jeśli jest HTN (True): wystarczy podnieść jeden z progów powyżej aktualnej wartości
    # - jeśli nie ma HTN (False): trzeba obniżyć oba progi do <= aktualnych wartości (bo AND)
    if diagnosed:
        # podnieś SBP_thr do > sbp albo DBP_thr do > dbp
        new_sbp_thr = int(np.floor(sbp) + 1)
        new_dbp_thr = int(np.floor(dbp) + 1)

        delta_sbp_thr = new_sbp_thr - sbp_thr
        delta_dbp_thr = new_dbp_thr - dbp_thr

        if delta_sbp_thr <= delta_dbp_thr:
            return {
                "diagnosed": True,
                "change": {
                    "param": "SBP_threshold",
                    "old": sbp_thr,
                    "new": new_sbp_thr,
                    "delta": int(delta_sbp_thr),
                    "message": (
                        f"Aby decyzja klasyczna zmieniła się na 'brak HTN' bez zmiany danych pacjenta, "
                        f"należałoby podnieść próg SBP z {sbp_thr} do {new_sbp_thr} (zmiana o +{int(delta_sbp_thr)})."
                    )
                }
            }
        else:
            return {
                "diagnosed": True,
                "change": {
                    "param": "DBP_threshold",
                    "old": dbp_thr,
                    "new": new_dbp_thr,
                    "delta": int(delta_dbp_thr),
                    "message": (
                        f"Aby decyzja klasyczna zmieniła się na 'brak HTN' bez zmiany danych pacjenta, "
                        f"należałoby podnieść próg DBP z {dbp_thr} do {new_dbp_thr} (zmiana o +{int(delta_dbp_thr)})."
                    )
                }
            }

    else:
        # żeby decyzja stała się True przy AND, oba progi muszą spaść do <= wartości pacjenta
        new_sbp_thr = int(np.ceil(sbp))
        new_dbp_thr = int(np.ceil(dbp))

        delta_sbp_thr = sbp_thr - new_sbp_thr
        delta_dbp_thr = dbp_thr - new_dbp_thr

        return {
            "diagnosed": False,
            "change": {
                "param": "SBP_threshold & DBP_threshold",
                "old": (sbp_thr, dbp_thr),
                "new": (new_sbp_thr, new_dbp_thr),
                "delta": (int(delta_sbp_thr), int(delta_dbp_thr)),
                "message": (
                    f"Aby decyzja klasyczna zmieniła się na 'HTN' bez zmiany danych pacjenta (reguła AND), "
                    f"należałoby obniżyć progi do: SBP {sbp_thr}->{new_sbp_thr} (o -{int(delta_sbp_thr)}), "
                    f"DBP {dbp_thr}->{new_dbp_thr} (o -{int(delta_dbp_thr)})."
                )
            }
        }

# =========================
# 2) Reguły rozmyte (fuzzy) – Mamdani
# =========================
def trimf(x, a, b, c):
    if x <= a or x >= c:
        return 0.0
    if a < x < b:
        return (x - a) / (b - a)
    if b <= x < c:
        return (c - x) / (c - b)
    return 0.0

def trapmf(x, a, b, c, d):
    if x <= a or x >= d:
        return 0.0
    if a < x < b:
        return (x - a) / (b - a)
    if b <= x <= c:
        return 1.0
    if c < x < d:
        return (d - x) / (d - c)
    return 0.0

# Uniwersa
SBP_U = np.linspace(80, 220, 701)
DBP_U = np.linspace(40, 140, 701)
RISK_U = np.linspace(0, 100, 1001)

# Zbiory rozmyte dla SBP/DBP (prosto, ale zgodnie z ideą dokumentu)
def mu_sbp_normal(x):     return trapmf(x, 80, 90, 120, 130)
def mu_sbp_border(x):     return trimf(x, 120, 135, 150)
def mu_sbp_high(x):       return trapmf(x, 140, 150, 220, 220)

def mu_dbp_normal(x):     return trapmf(x, 40, 50, 80, 85)
def mu_dbp_border(x):     return trimf(x, 80, 88, 96)
def mu_dbp_high(x):       return trapmf(x, 90, 95, 140, 140)

# Zbiory rozmyte dla RISK
def mu_risk_low(y):       return trapmf(y, 0, 0, 25, 40)
def mu_risk_med(y):       return trimf(y, 30, 50, 70)
def mu_risk_high(y):      return trapmf(y, 60, 75, 100, 100)

def centroid_defuzz(y_u, mu_u):
    denom = np.trapz(mu_u, y_u)
    if denom == 0:
        return 0.0
    return np.trapz(y_u * mu_u, y_u) / denom

def risk_label(crisp):
    if crisp < 35:
        return "low"
    if crisp < 65:
        return "medium"
    return "high"

def mamdani_infer(patient):
    sbp = patient["systolic_bp"]
    dbp = patient["diastolic_bp"]

    mu = {
        "sbp_normal": mu_sbp_normal(sbp),
        "sbp_border": mu_sbp_border(sbp),
        "sbp_high": mu_sbp_high(sbp),
        "dbp_normal": mu_dbp_normal(dbp),
        "dbp_border": mu_dbp_border(dbp),
        "dbp_high": mu_dbp_high(dbp),
    }

    # Reguły fuzzy (wariant nadciśnieniowy jako baza do kontrfakty)
    # R1: IF SBP is high OR DBP is high THEN risk is high
    r1 = max(mu["sbp_high"], mu["dbp_high"])

    # R2: IF SBP is border OR DBP is border THEN risk is medium
    r2 = max(mu["sbp_border"], mu["dbp_border"])

    # R3: IF SBP is normal AND DBP is normal THEN risk is low
    r3 = min(mu["sbp_normal"], mu["dbp_normal"])

    rules_fired = [
        {"rule": "FR1_high", "alpha": r1, "then": "risk_high"},
        {"rule": "FR2_med",  "alpha": r2, "then": "risk_medium"},
        {"rule": "FR3_low",  "alpha": r3, "then": "risk_low"},
    ]

    # Implikacja + agregacja po uniwersum RISK (min–max)
    mu_high = np.array([min(r1, mu_risk_high(y)) for y in RISK_U])
    mu_med  = np.array([min(r2, mu_risk_med(y))  for y in RISK_U])
    mu_low  = np.array([min(r3, mu_risk_low(y))  for y in RISK_U])

    mu_agg = np.maximum.reduce([mu_low, mu_med, mu_high])

    crisp = centroid_defuzz(RISK_U, mu_agg)

    debug = {
        "mu_inputs": mu,
        "rules": rules_fired,
        "mu_agg_max": float(np.max(mu_agg))
    }

    return crisp, mu_agg, debug


# =========================
# 3) Wyjaśnialność + kontrfakty (wariant 12)
# =========================
def rule_contributions_area(mu_agg_parts):
    # Przybliżenie wkładu reguł: pole pod przyciętym zbiorem / suma pól
    areas = {k: float(np.trapz(v, RISK_U)) for k, v in mu_agg_parts.items()}
    total = sum(areas.values()) if sum(areas.values()) > 0 else 1.0
    contrib = {k: areas[k] / total for k in areas}
    return areas, contrib

def explainable_fuzzy_with_counterfactual(patient, target_label="low"):
    sbp = patient["systolic_bp"]
    dbp = patient["diastolic_bp"]

    # Bazowe wnioskowanie
    crisp0, _, dbg0 = mamdani_infer(patient)
    label0 = risk_label(crisp0)

    # Do wyjaśnień liczymy osobno części agregacji
    mu = dbg0["mu_inputs"]
    r1 = max(mu["sbp_high"], mu["dbp_high"])
    r2 = max(mu["sbp_border"], mu["dbp_border"])
    r3 = min(mu["sbp_normal"], mu["dbp_normal"])

    mu_high = np.array([min(r1, mu_risk_high(y)) for y in RISK_U])
    mu_med  = np.array([min(r2, mu_risk_med(y))  for y in RISK_U])
    mu_low  = np.array([min(r3, mu_risk_low(y))  for y in RISK_U])

    areas, contrib = rule_contributions_area({
        "FR1_high": mu_high,
        "FR2_med":  mu_med,
        "FR3_low":  mu_low
    })

    rules_ranked = [
        {"rule": "FR1_high", "alpha": float(r1), "contribution": float(contrib["FR1_high"])},
        {"rule": "FR2_med",  "alpha": float(r2), "contribution": float(contrib["FR2_med"])},
        {"rule": "FR3_low",  "alpha": float(r3), "contribution": float(contrib["FR3_low"])},
    ]
    rules_ranked.sort(key=lambda x: x["contribution"], reverse=True)

    # Kontrfakty: minimalne obniżenie SBP/DBP aby osiągnąć target_label
    def target_ok(crisp):
        if target_label == "low":
            return crisp < 35
        if target_label == "medium":
            return crisp < 65
        return True

    best = None
    # przeszukiwanie: obniżamy SBP i DBP do pewnych granic, szukamy najmniejszej zmiany
    for dsbp in range(0, 61):   # do 60 mmHg
        for ddbp in range(0, 41):  # do 40 mmHg
            cand = dict(patient)
            cand["systolic_bp"] = max(80, sbp - dsbp)
            cand["diastolic_bp"] = max(40, dbp - ddbp)

            crisp_c, _, _ = mamdani_infer(cand)
            if target_ok(crisp_c):
                cost = dsbp + ddbp  # L1
                if (best is None) or (cost < best["cost"]):
                    best = {
                        "delta_sbp": dsbp,
                        "delta_dbp": ddbp,
                        "new_sbp": int(cand["systolic_bp"]),
                        "new_dbp": int(cand["diastolic_bp"]),
                        "new_crisp": float(crisp_c),
                        "new_label": risk_label(crisp_c),
                        "cost": cost
                    }
        # szybkie przycięcie: jak już znaleziono rozwiązanie z bardzo małym kosztem
        if best is not None and best["cost"] <= 2:
            break

    if best is None:
        cf_msg = "Nie znaleziono prostego kontrfakty w zadanym zakresie zmian."
    else:
        cf_msg = (
            f"Aby obniżyć wynik do poziomu '{target_label}', należałoby obniżyć "
            f"SBP o ~{best['delta_sbp']} (do {best['new_sbp']}) oraz DBP o ~{best['delta_dbp']} "
            f"(do {best['new_dbp']}). Oczekiwany wynik: {best['new_crisp']:.1f} ({best['new_label']})."
        )

    return {
        "risk_crisp": float(crisp0),
        "risk_label": label0,
        "mu_inputs": dbg0["mu_inputs"],
        "rules_ranked": rules_ranked,
        "counterfactual": {
            "target_label": target_label,
            "best": best,
            "message": cf_msg
        }
    }

def print_explanation(patient_id, exp):
    print(f"Pacjent: {patient_id}")
    print(f"Ryzyko (fuzzy): {exp['risk_crisp']:.1f} -> {exp['risk_label']}")
    print("Ranking reguł (wkład):")
    for r in exp["rules_ranked"]:
        print(f"  {r['rule']}: alpha={r['alpha']:.2f}, contribution={100*r['contribution']:.1f}%")
    print("Kontrfakty (wariant 12):")
    print(f"  {exp['counterfactual']['message']}")


# =========================
# 4) Uruchomienie dla wszystkich pacjentów + tabela wyników
# =========================
rows = []

for _, row in df.iterrows():
    p = row.to_dict()

    classic_out, classic_fired = expert_system_classic(p, classic_rules)
    cf_classic = counterfactual_classic_htn(p)
    cf_classic_thr = counterfactual_classic_threshold_change(p)

    exp_fuzzy = explainable_fuzzy_with_counterfactual(p, target_label="low")

    rows.append({
        "patient_id": p["patient_id"],
        "sbp": p["systolic_bp"],
        "dbp": p["diastolic_bp"],
        "classic_htn": bool(classic_out.get("Hypertension", 0) > 0),
        "classic_cf": cf_classic["suggestion"]["message"],
        "fuzzy_risk": exp_fuzzy["risk_crisp"],
        "fuzzy_label": exp_fuzzy["risk_label"],
        "fuzzy_top_rule": exp_fuzzy["rules_ranked"][0]["rule"],
        "fuzzy_cf": exp_fuzzy["counterfactual"]["message"],
    })

results_df = pd.DataFrame(rows)
results_df.sort_values(["fuzzy_risk"], ascending=False).head(10)


,patient_id,sbp,dbp,classic_htn,classic_cf,fuzzy_risk,fuzzy_label,fuzzy_top_rule,fuzzy_cf
26,P27,176,115,True,"Aby zmienić decyzję klasyczną na 'brak HTN', n...",83.436107,high,FR1_high,"Aby obniżyć wynik do poziomu 'low', należałoby..."
24,P25,173,113,True,"Aby zmienić decyzję klasyczną na 'brak HTN', n...",83.436107,high,FR1_high,"Aby obniżyć wynik do poziomu 'low', należałoby..."
22,P23,98,101,False,Decyzja klasyczna nie wskazuje HTN. Margines d...,83.436107,high,FR1_high,"Aby obniżyć wynik do poziomu 'low', należałoby..."
5,P06,155,105,True,"Aby zmienić decyzję klasyczną na 'brak HTN', n...",83.436107,high,FR1_high,"Aby obniżyć wynik do poziomu 'low', należałoby..."
20,P21,189,100,True,"Aby zmienić decyzję klasyczną na 'brak HTN', n...",83.436107,high,FR1_high,Nie znaleziono prostego kontrfakty w zadanym z...
12,P13,152,75,False,Decyzja klasyczna nie wskazuje HTN. Margines d...,83.436107,high,FR1_high,"Aby obniżyć wynik do poziomu 'low', należałoby..."
16,P17,188,61,False,Decyzja klasyczna nie wskazuje HTN. Margines d...,83.436107,high,FR1_high,Nie znaleziono prostego kontrfakty w zadanym z...
18,P19,121,105,False,Decyzja klasyczna nie wskazuje HTN. Margines d...,81.250080,high,FR1_high,"Aby obniżyć wynik do poziomu 'low', należałoby..."
29,P30,122,100,False,Decyzja klasyczna nie wskazuje HTN. Margines d...,79.386285,high,FR1_high,"Aby obniżyć wynik do poziomu 'low', należałoby..."
1,P02,148,98,True,"Aby zmienić decyzję klasyczną na 'brak HTN', n...",79.386285,high,FR1_high,"Aby obniżyć wynik do poziomu 'low', należałoby..."
